In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Filename

In [ ]:
name = 'file_0000.csv'

## Libraries

In [2]:
!pip install cchardet
!pip install requests-futures
!pip install http_request_randomizer
!pip install bs4
!pip install lxml

## Imports

In [3]:
from requests import Session
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from os import path, mkdir
import re
import pandas as pd
import time
import cchardet
import lxml
from numpy import random
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

# Scraping

In [ ]:
req_proxy = RequestProxy()  
proxy_list = req_proxy.get_proxy_list()

2021-05-25 07:14:51,515 http_request_randomizer.requests.useragent.userAgent INFO     Using local file for user agents: /usr/local/lib/python3.7/dist-packages/http_request_randomizer/requests/proxy/../data/user_agents.txt
2021-05-25 07:14:51,518 root   DEBUG    === Initialized Proxy Parsers ===
2021-05-25 07:14:51,520 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2021-05-25 07:14:51,521 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2021-05-25 07:14:51,523 root   DEBUG    	 SslProxy parser of 'https://www.sslproxies.org' with required bandwidth: '150' KBs
2021-05-25 07:14:51,525 root   DEBUG    =================================
2021-05-25 07:14:51,808 root   DEBUG    Added 300 proxies from FreeProxy
2021-05-25 07:14:52,558 http_request_randomizer.requests.parsers.PremProxyParser WARNING  Proxy Provider url failed: https://premproxy.com/list/
2021-05-25 07:14:52,560 http_reque

In [ ]:
table = pd.read_csv('/content/drive/MyDrive/Project/Data/datasets/' + name)
table = table[[colname for colname in table.columns if 'Unnamed:' not in colname]]

In [ ]:
scraped_path = '/content/drive/MyDrive/Project/Data/Scraped/'

if not path.exists(scraped_path):
  mkdir(scraped_path)

scraped_filename = scraped_path + name[:name.find('.csv')] + '_scraped.csv'

old_cols = table.columns.to_list() 

scrape_cols = ['image', 'has_video', 'story', 'risks', 'creator_bio', 'n_tiers', 'tiers_values', 'n_images', ' n_gifs', 
                             'n_websites', 'fb_linked', 'n_collab', 'collab_names']
cols = old_cols + scrape_cols
df = pd.DataFrame(columns=cols)

if path.exists(scraped_filename):
  df = pd.read_csv(scraped_filename)
  # cut table to restart after last index of previous run
  table = table[df.index.stop:]

# Scraping

In [ ]:
http_proxy  = None
https_proxy = None
proxy = None
proxyDict = {
              "http"  : http_proxy,
              "https" : https_proxy,
            }
start = time.time()
k = False
fs = FuturesSession()
sess = Session()
r = fs.get("https://www.kickstarter.com").result()
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]
headers= {
              "x-csrf-token": xcsrf
          }
query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

last_10 = []
c= 0

for i, row in table.iterrows():
  url = row['project_url']
  succesful = False
  page = time.time()
  slug = re.search('/projects/(.*)\?', url).group(1)
  url = re.search('(.*)\?', url).group(1)
  print(f"------ Page {i}: {slug} ------")

  while not succesful:
    try:
      rs = [fs.get(url, timeout=10), fs.post("https://www.kickstarter.com/graph", proxies = proxyDict,
          headers=headers,
          json = {
              "operationName":"Campaign",
              "variables":{
                  "slug": slug
              },
              "query": query
          }, timeout=25)]

      time.sleep(1.5)
      r = None
      r = sess.get(url + '/creator_bio')

      soup = BeautifulSoup(r.content, 'lxml')
      ## Websites
      websites = 0
      if soup.find("ul", {'class':'links list f5 bold'}):
        websites = len(soup.find("ul", {'class':'links list f5 bold'}).find_all('li'))

      ## Bio
      bio = soup.find('div', {'class': 'readability'})
      if bio:
        bio_text = []
        for p in bio.find_all('p'):
            if p.find(text=True) != None:
                bio_text.append(p.find(text=True))

        bio_text = ' '.join(bio_text)
      else:
        bio_text = None
      ## Facebook
      fb = soup.find("div", {'class':"facebook py2 border-bottom f5"})
      fb_linked = fb.find("a", {'class': 'popup'}) != None

      # Collaborators
      n_collab = 0
      collaborators = None
      if soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}):
        collaborators = soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}).findChildren('a')
        n_collab = len(collaborators)

      # names
      collab_names = []
      if collaborators:
        for col in collaborators:
            collab_names.append(re.search(r'/profile/(.*?)/about', str(col)).group(1))

      r = None
      r = rs[0].result()
      soup = BeautifulSoup(r.content, 'lxml')

      ## Image
      image = soup.find('img')['src']
      ## Video
      has_video = soup.find('video') is not None
      ## Pledge tiers
      tiers = soup.find_all("div", {"class": "pledge__info"})
      tiers_values = []
      for tier in tiers:
          s = str(tier.find("span", {"class": "money"}))
          if re.search(r'\d+', s):
            tiers_values.append(int(re.findall(r'\d+', s)[0]))
          else:
            tiers_values.append('0')
      n_tiers = len(tiers_values)

      r = None
      r = rs[1].result()
      result = r.json()

      story_html = result["data"]["project"]["story"]
      story = BeautifulSoup(story_html, 'html.parser')
      n_gifs = len(story.find_all('img', {'class': "fit js-lazy-image"}))
      n_images = len(story.find_all('img')) - n_gifs


      # text
      story_text = ' '.join([p for p in story.find_all(text=True) if i not in ['\n', ' ']])

      risks = result["data"]["project"]["risks"]

      df.loc[i] = pd.Series(table.loc[i].values.tolist() + [image, has_video, story_text, risks, bio_text, n_tiers, tiers_values, n_images, n_gifs, websites,
                      fb_linked, n_collab, collab_names], index = cols)
      succesful = True
      if time.time()-page < 2:
        time.sleep(2 - (time.time() - page))
      print('Time for this page was {}s'.format(round(time.time()-page, 2)))

      last_10.append(time.time()-page)
      if len(last_10) == 11:
        last_10.pop(0)
        if sum([x >= 5 for x in last_10]) >= 5:
          print('Proxy too slow')
          proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
          if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
          http_proxy  = 'http://' + proxy
          https_proxy = 'https://' + proxy
          proxyDict = {
                        "http"  : http_proxy,
                        "https" : https_proxy,
                      }

          last_10 = []

    except KeyboardInterrupt:
      k = True
      print('interrupted!')
      break

    except:
      if r and r.status_code == 200:
        time.sleep(1.5)
        print('Problems parsing, scraping skipped!!')
        df.loc[i] = pd.Series(table.loc[i].values.tolist(), index = old_cols)
        succesful =True
      elif r and r.status_code == 429:
        print("Too many requests, rotate ip")
        time.sleep(5)
        proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
        if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = {
                      "http"  : http_proxy,
                      "https" : https_proxy,
                    }

      else:
        print("Bad Proxy")
        time.sleep(5)
        proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
        if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = {
                      "http"  : http_proxy,
                      "https" : https_proxy,
                    }



  if k:
    break

  if i%500 == 0:
    print(f'\n\n ----- Reached Page {i}, saving dataframe to {scraped_filename} ----- \n\n')
    df.to_csv(scraped_filename, index=False)


print(f'Total time for {i - table.index.start} pages is {round(time.time()-start, 2)} seconds')

------ Page 14501: apesos/a-new-version-of-the-tarot ------
Time for this page was 2.09s
------ Page 14502: 1270322371/type-one-writers ------
Time for this page was 2.21s
------ Page 14503: mollieart/fantasy-and-surreal-postcards-and-prints ------
Time for this page was 2.27s
------ Page 14504: 1712814392/christmas-rush-watch-your-mouth-drop-an-f-bomb ------
Time for this page was 2.14s
------ Page 14505: umar-ditta/lad-the-homecoming-issue-2 ------
Time for this page was 2.37s
------ Page 14506: jamesstimpson/the-list-2-not-your-normal-christmas-tale-continued ------
Time for this page was 2.36s
------ Page 14507: 1395359268/united-intuitive-painting ------
Time for this page was 2.1s
------ Page 14508: 1576279164/journey-under-the-arctic-graphic-novel-signed-and-remarked ------
Time for this page was 2.93s
------ Page 14509: 2107613952/paper-clip-flowers ------
Time for this page was 2.01s
------ Page 14510: 1896022713/perils-of-superman-skit ------
Time for this page was 2.0s
-----

In [ ]:
df.to_csv(scraped_filename, index=False)